In [10]:
import os
from pathlib import Path
from glob import glob
from tqdm import tqdm

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import librosa
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB

In [14]:
class SpeechCommandsDataset(Dataset):
    def __init__(self, data_dir, target_sr=16000, transform=None, fixed_num_samples=16000):
        self.data_dir = data_dir
        self.target_sr = target_sr
        self.transform = transform
        self.fixed_num_samples = fixed_num_samples

        self.classes = sorted(
            d.name for d in os.scandir(data_dir) if d.is_dir()
        )
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        self.files = []
        for cls in self.classes:
            pattern = os.path.join(data_dir, cls, "*.wav")
            for path in glob(pattern):
                self.files.append((path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        path, label = self.files[idx]

        y, sr = librosa.load(path, sr=self.target_sr, mono=True)

        if len(y) > self.fixed_num_samples:
            y = y[:self.fixed_num_samples]
        elif len(y) < self.fixed_num_samples:
            pad = self.fixed_num_samples - len(y)
            y = np.pad(y, (0, pad), mode="constant")

        waveform = torch.from_numpy(y).float().unsqueeze(0)
        if self.transform:
            waveform = self.transform(waveform)

        return waveform, label

In [18]:
data_dir = 'datasets/speech_commands_split/'

mel_spectrogram = MelSpectrogram(
    sample_rate=16000,
    n_mels=64,
    f_min=20,
    f_max=8000
)
amp_to_db = AmplitudeToDB()

def waveform_to_logmel(waveform):
    mel = mel_spectrogram(waveform)
    logmel = amp_to_db(mel)
    return logmel

train_ds = SpeechCommandsDataset(os.path.join(data_dir, 'train'), transform=waveform_to_logmel)
val_ds = SpeechCommandsDataset(os.path.join(data_dir, 'val'), transform=waveform_to_logmel)
test_ds = SpeechCommandsDataset(os.path.join(data_dir, 'test'), transform=waveform_to_logmel)

batch_size = 64

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

num_classes = len(os.listdir(os.path.join(data_dir, 'train')))

In [4]:
class SelfAttentionBlock(nn.Module):
    def __init__(self, d_model=128, n_heads=7, d_ff=256, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, key_padding_mask=None):
        attn_out, _ = self.attn(
            x, x, x,
            key_padding_mask=key_padding_mask
        )
        x = self.norm1(x + self.dropout(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x

In [5]:
class CNNAttentionSpeechCommands(nn.Module):
    def __init__(self, n_classes, d_model=128, n_heads=4, d_ff=256, n_layers=2):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
        )

        self.proj = nn.Linear(64, d_model)

        self.layers = nn.ModuleList([
            SelfAttentionBlock(d_model, n_heads, d_ff)
            for _ in range(n_layers)
        ])

        self.classifier = nn.Linear(d_model, n_classes)

    def forward(self, x):
        x = self.cnn(x)
        B, C, Fp, Tp = x.shape

        x = x.mean(dim=2)
        x = x.permute(0, 2, 1)

        x = self.proj(x)

        for layer in self.layers:
            x = layer(x)
        
        x = x.mean(dim=1)

        logits = self.classifier(x)
        return logits

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for feats, labels in tqdm(loader, desc='Train:'):
        feats = feats.to(device)  
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(feats)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * feats.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for feats, labels in tqdm(loader, desc='Val:'):
        feats = feats.to(device)
        labels = labels.to(device)

        outputs = model(feats)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * feats.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

In [8]:
model = CNNAttentionSpeechCommands(
    n_classes=num_classes,
    d_model=128,
    n_heads=4,
    d_ff=256,
    n_layers=2
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [19]:
num_epochs = 20

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}')
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    print(f'    train_loss: {train_loss:.4f} acc: {train_acc:.4%}')
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f'    val_loss: {val_loss:.4f} acc: {val_acc:.2%}')

Epoch 1


Train:: 100%|██████████| 181/181 [00:14<00:00, 12.45it/s]


    train_loss: 0.6375 acc: 77.8558%


Val:: 100%|██████████| 39/39 [00:13<00:00,  2.81it/s]


    val_loss: 0.6939 acc: 75.97%
Epoch 2


...

    val_loss: 0.2409 acc: 92.86%
Epoch 20


Train:: 100%|██████████| 181/181 [00:16<00:00, 11.00it/s]


    train_loss: 0.1445 acc: 94.8911%


Val:: 100%|██████████| 39/39 [00:02<00:00, 14.88it/s]

    val_loss: 0.2613 acc: 92.45%


In [21]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for feats, labels in tqdm(test_loader):
        feats = feats.to(device)
        labels = labels.to(device)

        outputs = model(feats)
        loss = criterion(outputs, labels)

        _, preds = outputs.max(1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

100%|██████████| 39/39 [00:15<00:00,  2.56it/s]


In [22]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.90      0.97      0.93       354
           1       0.98      0.92      0.95       354
           2       0.86      0.94      0.90       354
           3       0.95      0.88      0.91       354
           4       0.93      0.93      0.93       354
           5       0.95      0.96      0.95       354
           6       0.94      0.92      0.93       354

    accuracy                           0.93      2478
   macro avg       0.93      0.93      0.93      2478
weighted avg       0.93      0.93      0.93      2478

